In [1]:
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [2]:
def load_data(data_dir="./data"):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset

In [3]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
def train_cifar(config, checkpoint_dir=None, data_dir=None):
    net = Net(config["l1"], config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
    print("Finished Training")

In [5]:
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [6]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 1, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))



# You can change the number of GPUs per trial here:
main(num_samples=10, max_num_epochs=10, gpus_per_trial=0)

Files already downloaded and verified
Files already downloaded and verified


2022-04-17 15:06:56,847	INFO trial_runner.py:803 -- starting train_cifar_eeb8a_00000
2022-04-17 15:06:58,713	INFO trial_runner.py:803 -- starting train_cifar_eeb8a_00001
2022-04-17 15:06:58,720	INFO trial_runner.py:803 -- starting train_cifar_eeb8a_00002
2022-04-17 15:06:58,725	INFO trial_runner.py:803 -- starting train_cifar_eeb8a_00003
2022-04-17 15:06:58,734	INFO trial_runner.py:803 -- starting train_cifar_eeb8a_00004
2022-04-17 15:06:58,784	INFO trial_runner.py:803 -- starting train_cifar_eeb8a_00005
2022-04-17 15:06:58,793	INFO trial_runner.py:803 -- starting train_cifar_eeb8a_00006
2022-04-17 15:06:58,800	INFO trial_runner.py:803 -- starting train_cifar_eeb8a_00007
2022-04-17 15:06:58,808	INFO trial_runner.py:803 -- starting train_cifar_eeb8a_00008
2022-04-17 15:06:58,816	INFO trial_runner.py:803 -- starting train_cifar_eeb8a_00009


== Status ==
Current time: 2022-04-17 15:06:56 (running for 00:00:00.21)
Memory usage on this node: 6.6/62.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 1.0/16 CPUs, 0/2 GPUs, 0.0/34.71 GiB heap, 0.0/17.35 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/priyammazumdar/ray_results/train_cifar_2022-04-17_15-06-56
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+---------------------+--------------+------+------+-------------+
| Trial name              | status   | loc                 |   batch_size |   l1 |   l2 |          lr |
|-------------------------+----------+---------------------+--------------+------+------+-------------|
| train_cifar_eeb8a_00000 | RUNNING  | 192.168.1.108:10421 |            2 |  128 |   32 | 0.000117108 |
| train_cifar_eeb8a_00001 | PENDING  |                     |            2 |   32 |    4 | 0.0240294   |
| tr

(func pid=10454) [1,  2000] loss: 2.311
(func pid=10465) [1,  2000] loss: 2.307
(func pid=10453) [1,  2000] loss: 2.328
(func pid=10457) [1,  2000] loss: 2.155
== Status ==
Current time: 2022-04-17 15:07:18 (running for 00:00:22.17)
Memory usage on this node: 11.8/62.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 10.0/16 CPUs, 0/2 GPUs, 0.0/34.71 GiB heap, 0.0/17.35 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/priyammazumdar/ray_results/train_cifar_2022-04-17_15-06-56
Number of trials: 10/10 (10 RUNNING)
+-------------------------+----------+---------------------+--------------+------+------+-------------+
| Trial name              | status   | loc                 |   batch_size |   l1 |   l2 |          lr |
|-------------------------+----------+---------------------+--------------+------+------+-------------|
| train_cifar_eeb8a_00000 | RUNNING  | 192.168.1.108:10421 |

(func pid=10451) [1,  6000] loss: 0.543
(func pid=10454) [1,  6000] loss: 0.771
(func pid=10465) [1,  6000] loss: 0.767
== Status ==
Current time: 2022-04-17 15:07:38 (running for 00:00:42.08)
Memory usage on this node: 11.8/62.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: -2.256511124420166
Resources requested: 10.0/16 CPUs, 0/2 GPUs, 0.0/34.71 GiB heap, 0.0/17.35 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/priyammazumdar/ray_results/train_cifar_2022-04-17_15-06-56
Number of trials: 10/10 (10 RUNNING)
+-------------------------+----------+---------------------+--------------+------+------+-------------+---------+------------+----------------------+
| Trial name              | status   | loc                 |   batch_size |   l1 |   l2 |          lr |    loss |   accuracy |   training_iteration |
|-------------------------+----------+---------------------+--------------+------+------+---------

...

== Status ==
Current time: 2022-04-17 15:19:12 (running for 00:12:15.59)
Memory usage on this node: 6.7/62.7 GiB
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 8.000: -1.2114240280002355 | Iter 4.000: -1.3481265175111592 | Iter 2.000: -1.605290038704872 | Iter 1.000: -2.28803605966568
Resources requested: 1.0/16 CPUs, 0/2 GPUs, 0.0/34.71 GiB heap, 0.0/17.35 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/priyammazumdar/ray_results/train_cifar_2022-04-17_15-06-56
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+-------------------------+------------+---------------------+--------------+------+------+-------------+---------+------------+----------------------+
| Trial name              | status     | loc                 |   batch_size |   l1 |   l2 |          lr |    loss |   accuracy |   training_iteration |
|-------------------------+------------+---------------------+--------------+------+------+-------------+---------+------------+----------------------|
| train_

== Status ==
Current time: 2022-04-17 15:19:32 (running for 00:12:35.62)
Memory usage on this node: 6.7/62.7 GiB
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 8.000: -1.2114240280002355 | Iter 4.000: -1.3481265175111592 | Iter 2.000: -1.605290038704872 | Iter 1.000: -2.28803605966568
Resources requested: 1.0/16 CPUs, 0/2 GPUs, 0.0/34.71 GiB heap, 0.0/17.35 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/priyammazumdar/ray_results/train_cifar_2022-04-17_15-06-56
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+-------------------------+------------+---------------------+--------------+------+------+-------------+---------+------------+----------------------+
| Trial name              | status     | loc                 |   batch_size |   l1 |   l2 |          lr |    loss |   accuracy |   training_iteration |
|-------------------------+------------+---------------------+--------------+------+------+-------------+---------+------------+----------------------|
| train_

== Status ==
Current time: 2022-04-17 15:19:52 (running for 00:12:55.66)
Memory usage on this node: 6.7/62.7 GiB
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 8.000: -1.2114240280002355 | Iter 4.000: -1.3481265175111592 | Iter 2.000: -1.605290038704872 | Iter 1.000: -2.28803605966568
Resources requested: 1.0/16 CPUs, 0/2 GPUs, 0.0/34.71 GiB heap, 0.0/17.35 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/priyammazumdar/ray_results/train_cifar_2022-04-17_15-06-56
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+-------------------------+------------+---------------------+--------------+------+------+-------------+---------+------------+----------------------+
| Trial name              | status     | loc                 |   batch_size |   l1 |   l2 |          lr |    loss |   accuracy |   training_iteration |
|-------------------------+------------+---------------------+--------------+------+------+-------------+---------+------------+----------------------|
| train_

2022-04-17 15:19:53,963	INFO tune.py:701 -- Total run time: 777.33 seconds (777.16 seconds for the tuning loop).


Result for train_cifar_eeb8a_00000:
  accuracy: 0.613
  date: 2022-04-17_15-19-53
  done: true
  experiment_id: 5feb8025c165493a984a69220e21f068
  hostname: workstation
  iterations_since_restore: 10
  loss: 1.0997307363914326
  node_ip: 192.168.1.108
  pid: 10421
  should_checkpoint: true
  time_since_restore: 775.132691860199
  time_this_iter_s: 66.6163740158081
  time_total_s: 775.132691860199
  timestamp: 1650226793
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: eeb8a_00000
  warmup_time: 0.0039730072021484375
  
== Status ==
Current time: 2022-04-17 15:19:53 (running for 00:12:57.17)
Memory usage on this node: 6.3/62.7 GiB
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 8.000: -1.2114240280002355 | Iter 4.000: -1.3481265175111592 | Iter 2.000: -1.605290038704872 | Iter 1.000: -2.28803605966568
Resources requested: 0/16 CPUs, 0/2 GPUs, 0.0/34.71 GiB heap, 0.0/17.35 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/priyammazumdar/ray_results/train